In [1]:
import pandas as pd

In [9]:
#read in 2015-2022 datasets and add year column
df_15 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2015.csv')
df_15["Year"] = 2015

df_16 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2016.csv')
df_16["Year"] = 2016

df_17 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2017.csv')
df_17["Year"] = 2017

df_18 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2018.csv')
df_18["Year"] = 2018

df_19 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2019.csv')
df_19["Year"] = 2019

df_20 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2020.csv')
df_20["Year"] = 2020

df_21 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2021.csv')
df_21["Year"] = 2021

df_22 = pd.read_csv('https://raw.githubusercontent.com/cklostermann/STAT-451-project/main/2022.csv')
df_22["Year"] = 2022

In [15]:
#merge datasets
df = pd.concat([df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22])

list(df.columns) #will have to clean up column names so that they can merge properly

['Country',
 'Region',
 'Happiness Rank',
 'Happiness Score',
 'Standard Error',
 'Economy (GDP per Capita)',
 'Family',
 'Health (Life Expectancy)',
 'Freedom',
 'Trust (Government Corruption)',
 'Generosity',
 'Dystopia Residual',
 'Year',
 'Lower Confidence Interval',
 'Upper Confidence Interval',
 'Happiness.Rank',
 'Happiness.Score',
 'Whisker.high',
 'Whisker.low',
 'Economy..GDP.per.Capita.',
 'Health..Life.Expectancy.',
 'Trust..Government.Corruption.',
 'Dystopia.Residual',
 'Overall rank',
 'Country or region',
 'Score',
 'GDP per capita',
 'Social support',
 'Healthy life expectancy',
 'Freedom to make life choices',
 'Perceptions of corruption',
 'Country name',
 'Regional indicator',
 'Ladder score',
 'Standard error of ladder score',
 'upperwhisker',
 'lowerwhisker',
 'Logged GDP per capita',
 'Ladder score in Dystopia',
 'Explained by: Log GDP per capita',
 'Explained by: Social support',
 'Explained by: Healthy life expectancy',
 'Explained by: Freedom to make life choi